home directory: cluster_doctor
function availability : ./kubectl/
1. get free nodes list
    - save it to list ./gitignored/free.txt
2. Get latest test results timmestamp from validation.db for all the free nodes in the free node list ( by accessing gcr-admin-pvc-access pod)
    - per node per test - latest timestamp
    - if a node has no test results - mark it as "never tested" - highest priority
3. Combine free nodes list with latest test results timestamps create a priority queue
    - Sort nodes by latest test results timestamps (oldest first - highest priority)
    - If the test results are not older than Z days - skip that node from the priority Queue
4. For each node in job queue list
    - Create a job to run cluster-doctor validation tests on that node
    - Submit the job to k8s cluster
    - Log job submission status

5. Monitor job status
    - if a job pending for more than X minutes - cancel the job and log the event
    - if a job runs longer than Y minutes - cancel the job
6. After job completion ( inside job pod)
    - Collect test results log
    - Update validation.db with new test results and timestamps
    - Log job completion status

7. Generate a daily report
    - Summary of nodes tested
    - Summary of test results
    - List of nodes that were never tested
    - Save report to ./gitignored/reports/daily_report_<date>.txt

In [ ]:
# variables
from symtable import Class
# import re # No longer needed

home_dir="/home/hari/b200/validation/cluster_doctor/"
functions_dir=home_dir+"kubectl/"


#Get free nodes list
# /home/hari/b200/validation/cluster_doctor/kubectl/cluster/freenode_list.sh

#create a cluster class

class Cluster:
    def __init__(self, ns):
        self.ns = ns
        #numerical timestamp
        self.timestamp = int(__import__('time').time())

    def freenodes(self):
        import subprocess
        # Using the new script that returns just the node names
        cmd = functions_dir+"cluster/freenode_list.sh"
        result = subprocess.run([cmd], stdout=subprocess.PIPE, shell=True)
        # Split by newline and filter empty strings
        nodes = [n for n in result.stdout.decode('utf-8').strip().split('\n') if n]
        return nodes

In [4]:
cluster=Cluster("gcr-admin",[])
print(cluster.freenodes())

['slc01-cl02-hgx-0001                            cap=  8  alloc=  8  used=  0  free=  8', 'slc01-cl02-hgx-0002                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0003                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0004                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0005                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0006                            cap=  8  alloc=  8  used=  0  free=  8', 'slc01-cl02-hgx-0007                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0008                            cap=  8  alloc=  8  used=  0  free=  8', 'slc01-cl02-hgx-0009                            cap=  8  alloc=  8  used=  0  free=  8', 'slc01-cl02-hgx-0010                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0011                            cap=  8  alloc=  8  used=  8  free=  0', 'slc01-cl02-hgx-0012